# # Main Scraper
# Execute the complete scraping workflow

## Load All Dependencies

In [1]:
%run config.ipynb
%run utils.ipynb
%run scraping_functions.ipynb

✓ Configuration loaded successfully
  - Default threads: 3
  - Max threads: 15
  - WebDriver timeout: 10s
✓ Configuration loaded successfully
  - Default threads: 3
  - Max threads: 15
  - WebDriver timeout: 10s
✓ Test URL: https://www.indeed.com/jobs?q=data+analyst&l=New+York+NY
✓ Utility functions loaded successfully
✓ Configuration loaded successfully
  - Default threads: 3
  - Max threads: 15
  - WebDriver timeout: 10s
✓ Configuration loaded successfully
  - Default threads: 3
  - Max threads: 15
  - WebDriver timeout: 10s
✓ Test URL: https://www.indeed.com/jobs?q=data+analyst&l=New+York+NY
✓ Utility functions loaded successfully
✓ Scraping functions loaded successfully
  - get_job_basic_info()
  - get_job_description()
  - process_job_with_description()


## User Input Section


In [2]:
# Job search parameters
job_title = input("Enter job title: ")
city = input("Enter city: ")
state = input("Enter state: ")

In [3]:
# Pagination settings
start_page_input = input("Enter starting page (0 for first page, 1 for second page, etc.): ")
start_page = int(start_page_input) if start_page_input.strip() else 0
print(f"Starting from page {start_page + 1}")

pages_input = input("Enter number of pages to scrape (default 1): ")
num_pages = int(pages_input) if pages_input.strip() else 1
print(f"Will scrape {num_pages} pages")


Starting from page 1
Will scrape 1 pages


In [4]:
# Threading settings
threads_input = input(f"Enter number of parallel threads (default {DEFAULT_THREADS}, max {MAX_THREADS}): ")
max_workers = int(threads_input) if threads_input.strip() else DEFAULT_THREADS
max_workers = min(max_workers, MAX_THREADS)
print(f"Using {max_workers} parallel threads")

Using 15 parallel threads


## Main Scraping Loop

In [5]:
# Generate initial URL
url = get_url(job_title, city, state, start_page)
print(f"\nSearch URL: {url}")


Search URL: https://www.indeed.com/jobs?q=Machine+Learning&l=san+jose+CA


In [6]:
# Setup WebDriver
driver = create_driver()

In [7]:
# Store all job records
records = []
next_page_url = None

try:
    for page_num in range(num_pages):
        current_page = start_page + page_num
        print(f"\n{'='*80}")
        print(f"SCRAPING PAGE {current_page + 1}")
        print(f"{'='*80}")
        
        # Navigate to URL
        if page_num == 0:
            driver.get(url)
        else:
            if next_page_url:
                driver.get(next_page_url)
            else:
                url = get_url(job_title, city, state, current_page)
                driver.get(url)
        
        # Wait for page to load
        time.sleep(random.randint(PAGE_LOAD_MIN, PAGE_LOAD_MAX))
        WebDriverWait(driver, WEBDRIVER_TIMEOUT).until(
            EC.presence_of_element_located((By.CLASS_NAME, "job_seen_beacon"))
        )
        
        # Find all job postings
        posts = driver.find_elements(By.CLASS_NAME, "job_seen_beacon")
        print(f"Found {len(posts)} jobs on page {current_page + 1}")
        
        # Phase 1: Collect basic info quickly
        print("\nPhase 1: Collecting basic job information...")
        job_basics = []
        for i, post in enumerate(posts):
            print(f"  Collecting job {i + 1}/{len(posts)}...", end="\r")
            basic_info = get_job_basic_info(post)
            if basic_info:
                job_basics.append(basic_info)
        
        print(f"\n✓ Collected {len(job_basics)} job listings")
        
        # Save next page URL before closing
        next_page_url = None
        if page_num < num_pages - 1:
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a[data-testid='pagination-page-next']")
                next_page_url = next_button.get_attribute("href")
                print(f"✓ Next page URL saved")
            except NoSuchElementException:
                print("⚠ No next page button found")
        
        # Close browser to avoid detection
        driver.quit()
        print("✓ Closed listing page browser")
        
        # Phase 2: Fetch descriptions in parallel
        print(f"\nPhase 2: Fetching job descriptions ({max_workers} parallel threads)...")
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_job = {
                executor.submit(process_job_with_description, job_data, i, len(job_basics)): job_data 
                for i, job_data in enumerate(job_basics)
            }
            
            for future in as_completed(future_to_job):
                try:
                    record = future.result()
                    records.append(record)
                except Exception as e:
                    safe_print(f"Error processing job: {e}")
        
        elapsed_time = time.time() - start_time
        print(f"\n✓ Completed {len(job_basics)} jobs in {elapsed_time:.2f} seconds")
        print(f"  Average: {elapsed_time/len(job_basics):.2f} seconds per job")
        print(f"\n✓ Total jobs collected so far: {len(records)}")
        
        # Create new driver for next page
        if page_num < num_pages - 1 and next_page_url:
            print("\nPreparing for next page...")
            driver = create_driver()
            time.sleep(random.randint(PAGE_SWITCH_MIN, PAGE_SWITCH_MAX))
        elif page_num < num_pages - 1:
            print("⚠ No next page available, stopping pagination")
            break

finally:
    try:
        driver.quit()
    except:
        pass


SCRAPING PAGE 1
Found 16 jobs on page 1

Phase 1: Collecting basic job information...
✓ Collected 16 job listings
✓ Closed listing page browser

Phase 2: Fetching job descriptions (15 parallel threads)...
[Thread] Processing job 1/16: Senior Machine Learning Engineer at TETRAMEM INC
[Thread] Processing job 2/16: Senior Machine Learning Engineer, Agentic at Robinhood
[Thread] Processing job 3/16: Member of Technical Staff, Machine Learning Engineer at Reinforce Labs, Inc.
[Thread] Processing job 4/16: Machine Learning Researcher / Engineer (Foundational Models) at Pathway
[Thread] Processing job 5/16: Machine Learning Engineer, ML Runtime & Optimization at pony.ai
[Thread] Processing job 6/16: Machine Learning Principal Architect at ASAP Talent Services
[Thread] Processing job 7/16: Sr. Machine Learning - Compiler Engineer III, AWS Neuron, Annapurna Labs at Amazon.com Services LLC
[Thread] Processing job 8/16: ML Engineer at Prospance Inc
[Thread] Processing job 9/16: Mathematical Scie

## Display Results Summary

In [8]:
print(f"\n{'='*80}")
print(f"SCRAPING COMPLETE")
print(f"{'='*80}")
print(f"Total jobs collected: {len(records)}")
print(f"\nFirst 3 jobs preview:")

for i, record in enumerate(records[:3], 1):
    print(f"\n--- Job {i} ---")
    print(f"Title: {record[0]}")
    print(f"Company: {record[1]}")
    print(f"Location: {record[2]}")
    print(f"Salary: {record[3]}")
    print(f"Description: {record[5][:150]}..." if len(record[5]) > 150 else f"Description: {record[5]}")

if len(records) > 3:
    print(f"\n... and {len(records) - 3} more jobs")

print(f"\n{'='*80}")
print("Next step: Run save_data.ipynb to export your data")
print(f"{'='*80}")


SCRAPING COMPLETE
Total jobs collected: 16

First 3 jobs preview:

--- Job 1 ---
Title: Sr. Machine Learning - Compiler Engineer III, AWS Neuron, Annapurna Labs
Company: Amazon.com Services LLC
Location: Cupertino, CA 95014
Salary: $151,300 - $261,500 a year
Description: The Product: AWS Machine Learning accelerators are at the forefront of AWS innovation and one of several AWS tools used for building Generative AI on ...

--- Job 2 ---
Title: AI Agent Engineer
Company: Tessera Labs
Location: San Jose, CA
Salary: $200,000 - $230,000 a year
Description: Job Summary
As we build to the next level, we’re looking for a top-quality AI engineer with a strong focus on AI agents - someone who knows how to lev...

--- Job 3 ---
Title: Machine Learning Principal Architect
Company: ASAP Talent Services
Location: Redwood City, CA
Salary: $215,000 - $265,000 a year
Description: Description
Location: San Francisco, Bay Area, Melo Park
ASAP Talent Services is a leading I.T. executive search firm. Our

## Store Records for Later Use
# This saves the 'records' variable so you can access it in other notebooks

In [9]:
%store records
print(f"\n✓ Stored {len(records)} records in Jupyter storage")
print("You can now access this in save_data.ipynb using: %store -r records")

Stored 'records' (list)

✓ Stored 16 records in Jupyter storage
You can now access this in save_data.ipynb using: %store -r records
